In [1]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb


In [2]:
#define resnet block
def ResidualXorBlock(lut1, lut2):
    main_path= bb.Sequential([
        bb.Convolution2d(lut1, filter_size=(3,3)),
        bb.Convolution2d(lut2, filter_size=(3,3))
    ])

    return bb.Xor([
        main_path, #output of conv layers
        bb.PassThrough()  #identity skip connection
    ])

In [3]:
# configuration
net_name              = 'MnistStochasticLutResnet'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist'
rtl_module_name       = 'MnistLutResnet'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')
epochs                = 4
mini_batch_size       = 32

# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

In [4]:
# define residual network(resnet)

lut_layer0_0 = bb.DifferentiableLut([192], batch_norm=False, binarize=False)
lut_layer0_1 = bb.DifferentiableLut([32], batch_norm=False, binarize=False)

lut_layer1_0 = bb.DifferentiableLut([192], batch_norm=False, binarize=False)
lut_layer1_1 = bb.DifferentiableLut([32], batch_norm=False, binarize=False)

lut_layer2_0 = bb.DifferentiableLut([256], batch_norm=False, binarize=False)
lut_layer2_1 = bb.DifferentiableLut([64], batch_norm=False, binarize=False)

lut_layer3_0 = bb.DifferentiableLut([256], batch_norm=False, binarize=False)
lut_layer3_1 = bb.DifferentiableLut([64], batch_norm=False, binarize=False)

lut_layer4_0 = bb.DifferentiableLut([1024], batch_norm=False, binarize=False)
lut_layer4_1 = bb.DifferentiableLut([420], batch_norm=False, binarize=False)
lut_layer4_2 = bb.DifferentiableLut([70], batch_norm=False, binarize=False)



out1 = lut_layer0_0.forward(in1, train=True)


#xor_result = ResidualXorBlock(lut_layer0_0, lut_layer1_0)

'''net = bb.Sequential([
            bb.Sequential([
                bb.Convolution2d(bb.Sequential([lut_layer0_0, lut_layer0_1]), filter_size=(3, 3)),
                bb.Convolution2d(bb.Sequential([lut_layer1_0, lut_layer1_1]), filter_size=(3, 3)),
                bb.StochasticMaxPooling(filter_size=(2, 2)),
            ]),
            bb.Sequential([
                bb.Convolution2d(bb.Sequential([lut_layer2_0, lut_layer2_1]), filter_size=(3, 3)),
                bb.Convolution2d(bb.Sequential([lut_layer3_0, lut_layer3_1]), filter_size=(3, 3)),
                bb.StochasticMaxPooling(filter_size=(2, 2)),
            ]),
            bb.Sequential([
                bb.Convolution2d(bb.Sequential([lut_layer4_0, lut_layer4_1, lut_layer4_2]),
                                    filter_size=(4, 4)),
            ]),
            bb.Reduce([10])
        ])
        '''


net.set_input_shape([1, 28, 28])

net.send_command("binary false")      
net.send_command("lut_binarize true")  

#print(net.get_info())


NameError: name 'in1' is not defined

In [6]:
#bb.load_networks(data_path, net)

# learning
loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

for epoch in range(epochs):
    loss.clear()
    metrics.clear()

    # learning
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
            
            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)

            metrics.calculate(y_buf, t_buf)
            net.backward(dy_buf)
            
            optimizer.update()

            t.set_postfix(loss=loss.get(), acc=metrics.get())
    
    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    bb.save_networks(data_path, net)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

NameError: name 'net' is not defined

In [9]:
# export verilog
with open(output_velilog_file, 'w') as f:
    f.write('\n`timescale 1ns / 1ps\n\n\n')
    f.write(bb.make_verilog_lut_cnv_layers(rtl_module_name + 'Cnv0', net[0]))
    f.write(bb.make_verilog_lut_cnv_layers(rtl_module_name + 'Cnv1', net[1]))
    f.write(bb.make_verilog_lut_cnv_layers(rtl_module_name + 'Cnv2', net[2]))

# Simulation用ファイルに上書きコピー
shutil.copyfile(output_velilog_file, sim_velilog_file)

# Simulationで使う画像の生成
def img_geneator():
    for data in dataset_test:
        yield data[0] # 画像とラベルの画像の方を返す

img = (bb.make_image_tile(480//28+1, 640//28+1, img_geneator())*255).astype(np.uint8)
bb.write_ppm(os.path.join(rtl_sim_path, 'mnist_test_160x120.ppm'), img[:,:120,:160])
bb.write_ppm(os.path.join(rtl_sim_path, 'mnist_test_640x480.ppm'), img[:,:480,:640])

NameError: name 'net' is not defined

In [10]:
# 学習したモデルを読み込み(念のため)
bb.load_networks(data_path, net)

# LUTモデルは BIT型を使ってメモリ節約が可能
bin_dtype = bb.DType.BIT  # bb.DType.BIT or bb.DType.FP32

# 同一形状のバイナリLUTを生成
bin_lut0_0 = bb.BinaryLut.from_sparse_model(lut_layer0_0, fw_dtype=bin_dtype)
bin_lut0_1 = bb.BinaryLut.from_sparse_model(lut_layer0_1, fw_dtype=bin_dtype)
bin_lut1_0 = bb.BinaryLut.from_sparse_model(lut_layer1_0, fw_dtype=bin_dtype)
bin_lut1_1 = bb.BinaryLut.from_sparse_model(lut_layer1_1, fw_dtype=bin_dtype)
bin_lut2_0 = bb.BinaryLut.from_sparse_model(lut_layer2_0, fw_dtype=bin_dtype)
bin_lut2_1 = bb.BinaryLut.from_sparse_model(lut_layer2_1, fw_dtype=bin_dtype)
bin_lut3_0 = bb.BinaryLut.from_sparse_model(lut_layer3_0, fw_dtype=bin_dtype)
bin_lut3_1 = bb.BinaryLut.from_sparse_model(lut_layer3_1, fw_dtype=bin_dtype)
bin_lut4_0 = bb.BinaryLut.from_sparse_model(lut_layer4_0, fw_dtype=bin_dtype)
bin_lut4_1 = bb.BinaryLut.from_sparse_model(lut_layer4_1, fw_dtype=bin_dtype)
bin_lut4_2 = bb.BinaryLut.from_sparse_model(lut_layer4_2, fw_dtype=bin_dtype)

# テスト用ネットワーク構築
frame_modulation_size = 7

test_net = bb.Sequential([
                bb.RealToBinary(frame_modulation_size=frame_modulation_size, bin_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut0_0, bin_lut0_1]), filter_size=(3, 3), fw_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut1_0, bin_lut1_1]), filter_size=(3, 3), fw_dtype=bin_dtype),
                bb.MaxPooling(filter_size=(2, 2), fw_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut2_0, bin_lut2_1]), filter_size=(3, 3), fw_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut3_0, bin_lut3_1]), filter_size=(3, 3), fw_dtype=bin_dtype),
                bb.MaxPooling(filter_size=(2, 2), fw_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut4_0, bin_lut4_1, bin_lut4_2]), filter_size=(4, 4), fw_dtype=bin_dtype),
                bb.Reduce([10], fw_dtype=bin_dtype),
                bb.BinaryToReal(frame_modulation_size=frame_modulation_size)
            ])
test_net.set_input_shape([1, 28, 28])

#print(test_net.get_info())

# 推論評価
test_loss    = bb.LossSoftmaxCrossEntropy()
test_metrics = bb.MetricsCategoricalAccuracy()

loss.clear()
metrics.clear()
for images, labels in tqdm(loader_test):
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = test_net.forward(x_buf, train=False)
    test_loss.calculate(y_buf, t_buf)
    test_metrics.calculate(y_buf, t_buf)

print('Binary LUT test : loss=%f accuracy=%f' % (test_loss.get(), test_metrics.get()))

NameError: name 'net' is not defined